# Introduction to Large Language Models (LLMs) for Research

This notebook is the first module of a workshop on using Large Language Models (LLMs) in research workflows. It focuses on:

- Setting up access to an LLM via the **Groq** API using the **OpenAI-compatible** Python client.
- Running your **first query** and understanding the request/response structure.
- Exploring the notion of **temperature** (randomness) in generation.
- Creating a **continuous conversation** by keeping your own chat history.

We adopt a **text-only** approach (no image/graph understanding) to ensure minimal setup and maximum reproducibility.

By the end of this notebook, you will be able to:
1. Configure a connection to the Groq API.
2. Send chat messages with the OpenAI-compatible client.
3. Control generation behaviour via `temperature`.
4. Maintain and reuse chat history for a continuing conversation.

## Background (very briefly)
An LLM is a probabilistic model over text. Given a sequence of tokens $x_{1:t}$, it assigns probabilities to the next token $x_{t+1}$. At inference, models sample from a distribution such as
$$p(x_{t+1}=i\mid x_{1:t}) = \mathrm{softmax}\!\left(\frac{z_i}{T}\right),$$
where $z_i$ is the logit for token $i$ and $T>0$ is the **temperature**. Lower $T$ concentrates probability mass on high-logit tokens (more deterministic), while higher $T$ spreads it out (more diverse). We will **demonstrate** this behaviour below.


## 1) Environment Setup
This section prepares the Python environment. If you encounter an `ImportError`, go back to 00_setup_guid notebook to install the package.

**What this cell does:**
- Imports the OpenAI modules.
- Does **not** make any external calls yet.


In [1]:
# If running outside the provided environment, uncomment the next line:
# %pip install openai
import os
from openai import OpenAI


## 2) Configure API Connection (Groq)
LLM APIs are **stateless** web services. We'll configure a client that speaks the OpenAI-compatible protocol, pointing it to Groq's base URL.

**What you'll do in this cell:**
1. Paste your Groq API key (to create an API key see 00_setup_guide notebook).
2. Set the base URL for Groq's OpenAI-compatible endpoint.
3. Instantiate the client.

**Notes:**
- Keep your API key private. In shared workshops, you can paste it, run this cell, and then clear the visible key.
- You can also store keys in environment variables or use a `.env` file if preferred.


In [ ]:
# Paste your Groq API key below (between quotes). 
# Note this is not a secure way of entering API key because it is visible to everyone that sees your notebook.  
os.environ["GROQ_API_KEY"] = ""  # <-- replace with your key.
model = "llama-3.3-70b-versatile" # Select your model https://console.groq.com/docs/models
os.environ["BASE_URL"] = "https://api.groq.com/openai/v1" # Groq uses an OpenAI-compatible API surface; we just change the base URL.


In [3]:
# Create the client
client = OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url=os.environ["BASE_URL"],
)
print("Groq client initialized!")


Groq client initialized!


## 3) First LLM Call — "Hello LLM" (Llama-3 70B)
Here we send a **single-turn** prompt with minimal scaffolding. The API expects a list of `messages`, where each message has a `role` and `content`.

**Roles:**
- `system`: high-level instructions (tone, persona, formatting).
- `user`: your question or instruction.
- `assistant`: the model's reply (the API returns this).

**What this cell does:**
- Creates a tiny conversation with `system` and `user` messages.
- Calls the model `llama3 70b` for higher-quality outputs compared to 8B.
- Prints the model's reply.

You can modify the user content and re-run to see different responses.


In [4]:
messages = [
    {"role": "system", "content": "You are a helpful research assistant. Be concise."},
    {"role": "user", "content": "Explain what a Large Language Model is in two sentences."}
]

response = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=0 # lower temperature -> more deterministic
)

print(response.choices[0].message.content)


A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, using complex algorithms to learn patterns and relationships within vast amounts of text data. By training on massive datasets, LLMs can generate human-like text, answer questions, and even engage in conversation, making them a powerful tool for various applications such as language translation, text summarization, and chatbots.


### Examining the response object

In [5]:
for str in response:
    print(str)

('id', 'chatcmpl-a772b541-683b-43d6-8d91-ab34bc331aa3')
('choices', [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, using complex algorithms to learn patterns and relationships within vast amounts of text data. By training on massive datasets, LLMs can generate human-like text, answer questions, and even engage in conversation, making them a powerful tool for various applications such as language translation, text summarization, and chatbots.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))])
('created', 1761507006)
('model', 'llama-3.3-70b-versatile')
('object', 'chat.completion')
('service_tier', 'on_demand')
('system_fingerprint', 'fp_55062f05af')
('usage', CompletionUsage(completion_tokens=86, prompt_tokens=57, total_tokens=143, completion_tokens_details=None, promp

In [6]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, using complex algorithms to learn patterns and relationships within vast amounts of text data. By training on massive datasets, LLMs can generate human-like text, answer questions, and even engage in conversation, making them a powerful tool for various applications such as language translation, text summarization, and chatbots.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))

## 4) Understanding `temperature`
The **temperature** parameter adjusts the **randomness** of token sampling. Intuitively, the model produces a probability distribution over possible next tokens from its logits $z$. The temperature rescales those logits:

$$p_i = \mathrm{softmax}\!\left(\frac{z_i}{T}\right) = \frac{\exp(z_i/T)}{\sum_j \exp(z_j/T)}.$$

- Lower $T$ (e.g. $T=0.2$): the distribution is *sharper* around high-probability tokens, yielding more **stable** outputs.
- Higher $T$ (e.g. $T=0.8$): the distribution is *flatter*, encouraging **diversity** and sometimes creativity.

**What this cell does:**
- Sends the *same* prompt twice, once with `temperature=0.01` and once with `temperature=0.9`.
- Prints both answers so you can compare tone and variability.


In [ ]:
prompt = "Describe the role of LLMs in academic research in one sentence."
for temp in [0.01, 0.9]:
    print()
    print("Temperature:", temp)
    for i in range(5):
        response = client.chat.completions.create(
            model=model,
            messages=prompt,
            temperature=temp # lower temperature -> more deterministic
        )
        print(response.choices[0].message.content)



Temperature: 0.01
A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, generating human-like text based on the input it receives. LLMs are trained on vast amounts of text data, allowing them to learn patterns and relationships in language, and can be used for tasks such as language translation, text summarization, and conversation generation.
A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, using complex algorithms and vast amounts of data to generate text, answer questions, and complete tasks. LLMs are trained on massive datasets of text from various sources, enabling them to learn patterns, relationships, and nuances of language, and generate human-like responses to a wide range of inputs and prompts.
A Large Language Model (LLM) is a type of artificial intelligence (AI) designed to process and understand human language, using complex algorith

## 5) Continuous Conversation (Keeping History)
LLM APIs do **not** keep state between calls. To build a conversation, you keep a list of messages and send the *entire* recent history each time. We'll implement a small helper that:

- Appends the user's message to a global `chat_history` list.
- Calls the model with that history.
- Appends the assistant's reply back into the history.
- Returns the latest reply for display.

We also keep the temperature low for focused answers. For longer chats, you can cap history to the last *k* turns to control token usage.


In [8]:
chat_history = []

def chat(user_input, temperature=0.2, max_turns=8):
    """Send one user turn and get a reply, preserving context.
    - Keeps system prompt + last `max_turns` user/assistant messages.
    """
    chat_history.append({"role": "user", "content": user_input})

    # Keep only the most recent `max_turns` pairs to control context size
    system = chat_history[:1]
    recent = chat_history[-(max_turns*2):] if len(chat_history) > 1 else []
    window = system + recent

    resp = client.chat.completions.create(
        model=model,
        messages=window,
        temperature=temperature,
    )
    reply = resp.choices[0].message.content
    chat_history.append({"role": "assistant", "content": reply})
    return reply


In [9]:
chat_history = [
    {"role": "system", "content": "Tailor your answers for a bioinformatician."}
]

chat("What is logistic regression?")
chat("How does it differ from linear regression?")

for ch in chat_history:
    print()
    print(ch)


{'role': 'system', 'content': 'Tailor your answers for a bioinformatician.'}

{'role': 'user', 'content': 'What is logistic regression?'}

{'role': 'assistant', 'content': "**Logistic Regression**\n=======================\n\nLogistic regression is a statistical method used for binary classification problems, where the goal is to predict a binary outcome (0/1, yes/no, etc.) based on one or more predictor variables. It is a widely used technique in bioinformatics, particularly in the analysis of high-throughput data, such as gene expression or genomic variation.\n\n**Mathematical Formulation**\n---------------------------\n\nLogistic regression models the probability of the positive class (e.g., disease presence) using a logistic function, also known as the sigmoid function:\n\np = 1 / (1 + e^(-z))\n\nwhere:\n\n* p is the probability of the positive class\n* e is the base of the natural logarithm\n* z is a linear combination of the predictor variables: z = β0 + β1*x1 + β2*x2 + … + βn\\*

In [10]:
chat("what are some other classification methods?")

for ch in chat_history:
    print()
    print(ch)


{'role': 'system', 'content': 'Tailor your answers for a bioinformatician.'}

{'role': 'user', 'content': 'What is logistic regression?'}

{'role': 'assistant', 'content': "**Logistic Regression**\n=======================\n\nLogistic regression is a statistical method used for binary classification problems, where the goal is to predict a binary outcome (0/1, yes/no, etc.) based on one or more predictor variables. It is a widely used technique in bioinformatics, particularly in the analysis of high-throughput data, such as gene expression or genomic variation.\n\n**Mathematical Formulation**\n---------------------------\n\nLogistic regression models the probability of the positive class (e.g., disease presence) using a logistic function, also known as the sigmoid function:\n\np = 1 / (1 + e^(-z))\n\nwhere:\n\n* p is the probability of the positive class\n* e is the base of the natural logarithm\n* z is a linear combination of the predictor variables: z = β0 + β1*x1 + β2*x2 + … + βn\\*

## 8) Exercise — Your First Prompt
Try your own research-related prompts. A few ideas:

1. Summarise your current project in **one paragraph**.
2. Ask for **three open research questions** in your field.
3. Request a **draft methods paragraph** describing your dataset and analysis steps.

Remember you can tweak `temperature` to trade off consistency vs creativity.


In [11]:
chat_history = [
    {"role": "system", "content": "Tailor your answers for a bioinformatician."}
]

In [12]:
# Example: replace with your own question(s)
question = "Summarise the challenges in renewable energy policy research."
print(chat(question))


As a bioinformatician, you're likely familiar with complex systems and data-driven approaches. Renewable energy policy research presents several challenges that can be broken down into the following categories:

1. **Integration and Interoperability**: Renewable energy sources, such as solar and wind power, have variable output, making it challenging to integrate them into existing energy grids. This requires advanced forecasting and grid management systems to ensure a stable and reliable energy supply.
2. **Data Quality and Availability**: High-quality data on energy production, consumption, and grid operations is essential for informed policy decisions. However, data gaps, inconsistencies, and lack of standardization can hinder research and policy development.
3. **Complexity and Uncertainty**: Renewable energy systems involve complex interactions between technological, economic, social, and environmental factors. Uncertainties, such as future energy demand, technology costs, and pol

## 9) Wrap-Up & Next Steps
- You configured an OpenAI-compatible client to talk to **Groq**.
- You sent your first prompts using **Llama-3 70B** and explored the impact of `temperature`.
- You kept conversation state locally in a Python list and learned how to save it.

**Key terms:** tokens, temperature, logits, softmax, stateless API, chat history.
